In [ ]:
from PropertyInvestment import PropertyInvestment
from StockInvestment import StockInvestment
from plotting import PlotSeries, plot


TIME_FRAME_IN_YEARS = 50

rent_per_square_meter = 14.49
price_per_square_meter = 4130
area = 157
property_rent = area * rent_per_square_meter
property_price = area * price_per_square_meter

seriesList = []
for house_purchase_delay in range(0, 60, 10):
    series = PlotSeries(
        "House purchase after " + str(house_purchase_delay) + " Years"
        if house_purchase_delay > 0
        else "House purchase immediately"
    )
    stock_inv = StockInvestment(0.1, 50000)
    prop_inv = None
    for i in range(TIME_FRAME_IN_YEARS * 12 + 2):
        budget = 2500
        series.put_value(
            stock_inv.get_value()
            + (prop_inv.get_total_amortization() if prop_inv is not None else 0)
        )
        if i == house_purchase_delay * 12:
            available_money = stock_inv.get_value() + budget - property_rent
            if available_money >= property_price:
                prop_inv = PropertyInvestment(property_price, 0.1, 0.1, property_price)
                stock_inv.put_deposit(budget - property_rent - property_price)
            else:
                prop_inv = PropertyInvestment(
                    property_price, 0.036, 0.01, available_money
                )
                stock_inv.put_deposit(-1 * stock_inv.get_value())
        elif prop_inv is None:
            stock_inv.put_deposit(budget - property_rent)
        else:
            stock_inv.put_deposit(budget - prop_inv.get_rate())
        stock_inv.step()
        if prop_inv is not None:
            prop_inv.step()
    seriesList.append(series)

plot(
    {
        "title": "Value over Time",
        "xlabel": "Year",
        "ylabel": "Value in Mio €",
        "series": seriesList,
    }
)